In [1]:
import datetime
import requests
import pandas as pd

In [2]:
companies = ['RELIANCE', 'BAJAJFINSV', 'HDFCBANK', 'HDFC', 'ITC', 'INFY', 'ADANIPORTS', 'ICICIBANK', 'BAJFINANCE', 'TCS', 'KOTAKBANK', 'LT', 'AXISBANK', 'MARUTI', 'SBIN', 'TATAMOTORS', 'TATASTEEL', 'HINDALCO', 'BHARTIARTL', 'NTPC', 'EICHERMOT', 'APOLLOHOSP', 'M&N', 'INDUSINDBK', 'ASIANPAINT', 'HINDUNILVR', 'ONGC', 'BAJAJ-AUTO', 'HEROMOTOCO', 'TITAN', 'WIPRO', 'HDFCLIFE', 'SHREECEM', 'ULTRACEMCO', 'TECHM', 'SUNPHARMA', 'CIPLA', 'HCLTECH', 'TATACONSUM', 'BPCL', 'UPL', 'SBILIFE', 'JSWSTEEL', 'POWERGRID', 'DIVISLAB', 'COALINDIA', 'GRASIM', 'DRREDDY', 'BRITANNIA', 'MESTLEIND']

In [3]:
# set up today and yesterday
current_date = datetime.date.today()
today = current_date.strftime("%d%m%Y")
# today = '14102022'


yesterday = current_date - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%d%m%Y')
# yesterday = '13102022'


In [ ]:
# source file name and base url for the source url
source_file_name = 'sec_bhavdata_full_{}.csv'
base_url = r'https://archives.nseindia.com/products/content/'

In [ ]:
main_bhav_data = pd.read_csv('nse_data_file.csv')

In [ ]:
main_bhav_data

In [ ]:
# source url to fetch data from nseIndia.com for new data.
source_url = base_url+source_file_name.format(today)

# setup the column names of the new file
column_names = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'ClosePrice'+today, 'AvgPrice', 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'NoOfTrades', 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]

# fetch today's bhav data file from nseindia.com
today_bhav_data = pd.read_csv(source_url, names=column_names)

# filtering the today's data
today_bhav_data = today_bhav_data.drop(columns=['Date','PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'AvgPrice', 'NoOfTrades'])
today_bhav_data = today_bhav_data.loc[today_bhav_data['Symbol'].isin(companies)]

company_count = {}
for company in companies:
    company_count[company] = 0

is_valid = [0 for _ in today_bhav_data.index]

today_bhav_data.insert(1, 'is_valid', is_valid)

for ind in today_bhav_data.index:
    company = today_bhav_data.at[ind, 'Symbol']
    if not company_count[company]:
        company_count[company] = 1
        today_bhav_data.at[ind, 'is_valid'] = 1

today_bhav_data = today_bhav_data.loc[today_bhav_data['is_valid'] == 1]

today_bhav_data.to_csv('temp_nse_data.csv', index=False)

In [ ]:
today_bhav_data = pd.read_csv('temp_nse_data.csv')
today_bhav_data

In [ ]:
# setup the column names for yesterday's data from the main bhav data file.
header_main = ['ClosePrice'+yesterday, 'TotalTradedQuantity'+yesterday, 'TurnoverInLacs'+yesterday, 'DeliveryQuantity'+yesterday, 'DeliveryPercentage'+yesterday]
header_main

# setup the column names for today's data from the nseindia.com
header_new = ['ClosePrice'+today, 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]
header_new


# extract the new data and paste to the main bhav data file.
for i in range(5):
    new_head = header_new[i]
    prev_head = main_bhav_data.columns.get_loc(header_main[i]) + 1

    extracted_col = today_bhav_data[new_head]
    main_bhav_data.insert(prev_head, new_head, extracted_col)

In [ ]:
main_bhav_data

In [ ]:
main_bhav_data.to_csv('nse_data_file.csv', index=False)